# Data set up 

## Load existing dataset

In [1]:
import pandas as pd
df = pd.read_csv('tournesol_scores_above_20_2024-03-15.csv')

## Create dataset

Download videos scores

In [12]:
import io
import zipfile

import requests
import pandas as pd

response = requests.get("https://api.tournesol.app/exports/all")
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
collective_scores = pd.read_csv(zip_file.open("collective_criteria_scores.csv"))

Requests youtube metadata : publication date, title and channel

In [14]:
from dataset import build_dataset
df = build_dataset(collective_scores) #only keeps videos with a tournesol score above 20

Save the dataframe

In [16]:
import datetime
df.to_csv('tournesol_scores_above_20_'+datetime.date.today().isoformat()+'.csv')


# Uniform Recommendation

Test parameters

In [2]:
bundle_size = 4
recent_videos_proportion = 0.75

recent_videos_to_sample = int(bundle_size * recent_videos_proportion)
old_videos_to_sample = int(bundle_size * (1 - recent_videos_proportion))

recent_videos_max_age_in_days = 21

Computes the age of each video

In [3]:
import datetime

from utils import get_age_in_days

today = datetime.datetime.today()
df['age_in_days'] = df.apply(lambda x: get_age_in_days(x, today), axis="columns")

Uniformly sample recent and old videos

In [4]:
recent_videos_sample = df.loc[df['age_in_days']<=21].sample(n=recent_videos_to_sample, replace=False)
old_videos_sample = df.loc[df['age_in_days']<=21].sample(n=old_videos_to_sample, replace=False)

uniform_sample = pd.concat([recent_videos_sample, old_videos_sample])

Download thumbnails in the thumbnails directory

In [5]:
import io

import requests
import ipywidgets as widgets

thumbnail_list = []

for id in uniform_sample['video']:
    thumbnail_url = "https://i.ytimg.com/vi/" + id + "/mqdefault.jpg"
    response = requests.get(thumbnail_url)
    open('thumbnails/'+id+'.jpg', 'wb').write(response.content)

Display images

In [8]:
from utils import make_box_for_grid

boxes = []
pd.set_option('display.max_colwidth', 999)
for video_id in uniform_sample['video']:
    video_title = uniform_sample.loc[uniform_sample['video']==video_id, 'title'].to_string(index=False)
    video_channel = uniform_sample.loc[uniform_sample['video']==video_id, 'channel'].to_string(index=False)
    
    file = open('thumbnails/'+video_id+'.jpg', 'rb')
    image = widgets.Image(value=file.read())
    image.layout.object_fit = 'contain'  
    
    boxes.append(make_box_for_grid(image, video_title, video_channel))

hbox_layout = widgets.Layout()
hbox_layout.width = '100%'
hbox_layout.justify_content = 'space-around'

hb = widgets.HBox()
hb.layout = hbox_layout
hb.children = boxes
hb



🦏 PATRON INCOGNITO: EMPLOYÉS PIÉGÉS OU PDG RIDICULISÉS ?
